In [4]:
import numpy as np
import pandas as pd 
import os
from pandas.tseries.holiday import USFederalHolidayCalendar

import utils
# from utils import load_data, get_train_val_split, get_stratified_splitter
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedGroupKFold, train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, make_scorer

from lightgbm import LGBMRegressor
from scipy.stats import kstest, kruskal, mannwhitneyu
from itertools import combinations
from collections import defaultdict
from tqdm import tqdm

import optuna
import pickle

In [2]:
import importlib
importlib.reload(utils)

<module 'utils' from 'C:\\Users\\johns\\Desktop\\probstats2\\EnergyPrediction-ASHRAE\\code\\utils.py'>

In [3]:
data_dict = utils.load_data('ashrae-energy-prediction')

Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.02 MB
Decreased by 73.88%


C:\Users\johns\Desktop\probstats2\EnergyPrediction-ASHRAE\code\utils.py:143: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  weather_train['timestamp'] = pd.to_datetime(weather_train['timestamp'], infer_datetime_format = True, utc = True).astype('datetime64[ns]')


Memory usage of dataframe is 9.60 MB
Memory usage after optimization is: 3.07 MB
Decreased by 68.05%
Memory usage of dataframe is 19.04 MB
Memory usage after optimization is: 5.13 MB
Decreased by 73.04%


C:\Users\johns\Desktop\probstats2\EnergyPrediction-ASHRAE\code\utils.py:151: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  train['timestamp'] = pd.to_datetime(train['timestamp'], infer_datetime_format = True, utc = True).astype('datetime64[ns]')


Memory usage of dataframe is 616.95 MB
Memory usage after optimization is: 289.19 MB
Decreased by 53.12%
Memory usage of dataframe is 1272.51 MB
Memory usage after optimization is: 358.53 MB
Decreased by 71.82%


C:\Users\johns\Desktop\probstats2\EnergyPrediction-ASHRAE\code\utils.py:166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[(train['site_id'] == 0) & (train['meter'] == 0)]['meter_reading'] = 0.2931 * train[(train['site_id'] == 0) & (train['meter'] == 0)]['meter_reading']


In [4]:
# Add weather features 
weather_features = ['cloud_coverage', 'dew_temperature', 'air_temperature', 
                    'sea_level_pressure', 'wind_direction', 'wind_speed', 'precip_depth_1_hr',]

hourly_by_site = data_dict["X_train"].groupby(['hour', 'month', 'site_id'])[weather_features].mean().reset_index()

data_dict["X_train"] = data_dict["X_train"].merge(
    hourly_by_site, 
    on=['hour', 'month', 'site_id'], 
    how='left', 
    suffixes=(None, '_hourly_by_site')
)

del hourly_by_site

for feature in weather_features:
    # Fill in NA values from weather with hourly by site columns 
    data_dict["X_train"][feature].fillna(
        data_dict["X_train"][feature + "_hourly_by_site"],
        inplace=True
    )
    
    # Fill in the rest with the median 
    data_dict["X_train"][feature].fillna(
        data_dict["X_train"][feature].median(),
        inplace=True
    )
    
    data_dict["X_train"][feature + "_diff_hourly_from_mean"] = data_dict["X_train"][feature] - \
        data_dict["X_train"][feature + "_hourly_by_site"]
    
data_dict["X_train"] = data_dict["X_train"].drop(columns = [feat + "_hourly_by_site" for feat in weather_features])

In [5]:
# Fill in NA with median values for floor count and year_built
for feature in ['year_built', 'floor_count']:
    data_dict["X_train"][feature].fillna(
        data_dict["X_train"][feature].median(), 
        inplace=True
    )

In [6]:
data_dict.keys()

dict_keys(['weather_test', 'X_train', 'X_test', 'y_train'])

## Examine Differences (Non-Parametric)
Using Bonferonni's Correction

### Milestone 2. 
- Show difference in sites across meter readings 
- get average meter reading per day per site 
- conduct a (non-parametric ANOVA) KS OR pairs (mann-whitney) to show that they are diff 
- train a model per site id (with rudimentary hyperparameter tuning) 
- John sites 0-7, Sharad sites 8-15 

### Milestone 3. Determine, per site, which primary uses are similar (if they have only a few buildings), which are diff
- for a given primary use, if diff, identify "clusters" of buildings that are similar 

In [2]:
features = ['year_built', 'floor_count', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'sea_level_pressure', 'wind_direction', 'wind_speed',
       'air_temperature_mean_lag7', 'air_temperature_max_lag7',
       'air_temperature_min_lag7', 'air_temperature_std_lag7',
       'cloud_coverage_mean_lag7', 'cloud_coverage_max_lag7',
       'cloud_coverage_min_lag7', 'cloud_coverage_std_lag7',
       'dew_temperature_mean_lag7', 'dew_temperature_max_lag7',
       'dew_temperature_min_lag7', 'dew_temperature_std_lag7',
       'precip_depth_1_hr_mean_lag7', 'precip_depth_1_hr_max_lag7',
       'precip_depth_1_hr_min_lag7', 'precip_depth_1_hr_std_lag7',
       'sea_level_pressure_mean_lag7', 'sea_level_pressure_max_lag7',
       'sea_level_pressure_min_lag7', 'sea_level_pressure_std_lag7',
       'wind_direction_mean_lag7', 'wind_direction_max_lag7',
       'wind_direction_min_lag7', 'wind_direction_std_lag7',
       'wind_speed_mean_lag7', 'wind_speed_max_lag7', 'wind_speed_min_lag7',
       'wind_speed_std_lag7', 'log_square_feet', 'weekday', 'hour', 'day',
       'weekend', 'month', 'primary_use_enc']

In [3]:
len(features)

44

In [8]:
def run_optuna_search_cv(
    site: int,
    meter: int,
    X_train: pd.DataFrame,
    y_train: pd.DataFrame,
    features: list,
    n_trials: int = 50,
    
):
    """
    Runs Optuna Search for LGBMRegressor
    """
    SITE_FILTER = X_train["site_id"] == site
    METER_FILTER = X_train["meter"] == meter
    
    X, y = X_train.loc[SITE_FILTER&METER_FILTER, features], y_train[SITE_FILTER&METER_FILTER]
    if X.shape[0] == 0:
        return None

    splitter_gen = utils.get_stratified_splitter(X_train[SITE_FILTER&METER_FILTER], y_train[SITE_FILTER&METER_FILTER])

    

    regressor = LGBMRegressor()

    param_distributions = {
        "max_depth": optuna.distributions.IntDistribution(-1, len(data_dict['X_train'].columns)),
        "num_leaves": optuna.distributions.IntDistribution(5, 50),
        "learning_rate": optuna.distributions.FloatDistribution(1e-7, 1, log=True),
        "n_estimators": optuna.distributions.IntDistribution(1, 300),
        "reg_alpha": optuna.distributions.FloatDistribution(1e-7, 1e7, log=True),
        "reg_lambda": optuna.distributions.FloatDistribution(1e-7, 1e7, log=True),            
    }

    """
    Parameters not searched over: 
    subsample_for_bin: int = 200000,
    min_split_gain: float = 0.0,
    min_child_weight: float = 0.001,
    min_child_samples: int = 20,
    subsample: float = 1.0,
    subsample_freq: int = 0,
    colsample_bytree: float = 1.0,
    random_state: Union[int, numpy.random.mtrand.RandomState, NoneType] = None,
    n_jobs: int = -1,
    """
    def rmse(estimator, X_test, y_test):
        y_pred = estimator.predict(X_test)
        return -1 * mean_squared_error(y_test, y_pred, squared=False)
    
    optuna_search = optuna.integration.OptunaSearchCV(
        regressor, 
        param_distributions,
        n_trials=n_trials,
        cv = splitter_gen,
        random_state=0, # IMPORTANT,
        refit=True,
        n_jobs=4,
        scoring = rmse,
        verbose=0
    )

    optuna_search.fit(X, y)
    y_pred = optuna_search.predict(X)
    
    return optuna_search

In [9]:
# This fits sites 0-7
N_TRIALS = 50
models = defaultdict(dict)

for site in tqdm(range(8)):
    for meter in range(4):
        
        optuna_search = run_optuna_search_cv(site, meter, data_dict["X_train"], data_dict["y_train"], features, N_TRIALS)
        
        model_identifier = "_".join([str(site), str(meter)])
        models[model_identifier] = optuna_search

  0%|                                                                                            | 0/8 [00:00<?, ?it/s]C:\Users\johns\AppData\Local\Temp\ipykernel_5792\659673755.py:51: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(
C:\Users\johns\anaconda3\envs\probstats\lib\site-packages\sklearn\model_selection\_split.py:909: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
[I 2023-04-08 21:44:51,324] A new study created in memory with name: no-name-490eba39-57bd-4d85-b81c-b45735de58b6
[I 2023-04-08 21:45:32,982] Trial 2 finished with value: -1.345808815239991 and parameters: {'max_depth': 39, 'num_leaves': 29, 'learning_rate': 1.8035122710050434e-06, 'n_estimators': 43, 'reg_alpha': 1310.7119392056818, 'reg_lambda': 2068.2275095410464}. Best is trial 2 with value: -1.345808815239991.
[I 2023-04-08 21

[I 2023-04-08 21:54:36,752] Trial 25 finished with value: -0.982925132947899 and parameters: {'max_depth': 46, 'num_leaves': 38, 'learning_rate': 0.23765322590623222, 'n_estimators': 106, 'reg_alpha': 0.01795350058962178, 'reg_lambda': 3332.515910947464}. Best is trial 21 with value: -0.9800276924140099.
[I 2023-04-08 21:54:52,241] Trial 24 finished with value: -0.9644114791203054 and parameters: {'max_depth': 43, 'num_leaves': 39, 'learning_rate': 0.15639877119149315, 'n_estimators': 162, 'reg_alpha': 0.03812814519574558, 'reg_lambda': 8132.218437949613}. Best is trial 24 with value: -0.9644114791203054.
[I 2023-04-08 21:54:57,565] Trial 27 finished with value: -1.130525576038727 and parameters: {'max_depth': 46, 'num_leaves': 26, 'learning_rate': 0.21327319860378344, 'n_estimators': 124, 'reg_alpha': 6.240270788079808, 'reg_lambda': 3.1759107882489057}. Best is trial 24 with value: -0.9644114791203054.
[I 2023-04-08 21:55:04,826] Trial 26 finished with value: -1.0155017753823827 and 

C:\Users\johns\AppData\Local\Temp\ipykernel_5792\659673755.py:51: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(
C:\Users\johns\anaconda3\envs\probstats\lib\site-packages\sklearn\model_selection\_split.py:909: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
[I 2023-04-08 22:02:22,026] A new study created in memory with name: no-name-43a5b610-e62f-4ed7-816a-58db6bc93c55
[I 2023-04-08 22:02:33,803] Trial 2 finished with value: -2.4061070980620327 and parameters: {'max_depth': 5, 'num_leaves': 31, 'learning_rate': 3.677419203827511e-05, 'n_estimators': 286, 'reg_alpha': 235116.29255904845, 'reg_lambda': 2.2165242507340274e-05}. Best is trial 2 with value: -2.4061070980620327.
[I 2023-04-08 22:02:42,543] Trial 3 finished with value: -2.3951750272953136 and parameters: {'max_depth': 29, 'num_leaves': 36, '

[I 2023-04-08 22:05:26,982] Trial 25 finished with value: -2.257257912098953 and parameters: {'max_depth': 50, 'num_leaves': 28, 'learning_rate': 0.1984956225296615, 'n_estimators': 38, 'reg_alpha': 10.707248018317197, 'reg_lambda': 66325.51596097056}. Best is trial 8 with value: -2.239951599133442.
[I 2023-04-08 22:05:31,398] Trial 26 finished with value: -2.2660946816755443 and parameters: {'max_depth': 51, 'num_leaves': 26, 'learning_rate': 0.2088108478679834, 'n_estimators': 51, 'reg_alpha': 30.133900869767793, 'reg_lambda': 68545.44016655705}. Best is trial 8 with value: -2.239951599133442.
[I 2023-04-08 22:05:35,925] Trial 27 finished with value: -2.272107926457301 and parameters: {'max_depth': 50, 'num_leaves': 26, 'learning_rate': 0.2792115195236119, 'n_estimators': 49, 'reg_alpha': 95.9448868036309, 'reg_lambda': 88242.36182308513}. Best is trial 8 with value: -2.239951599133442.
[I 2023-04-08 22:05:37,491] Trial 28 finished with value: -2.25963108829977 and parameters: {'max_

 12%|██████████                                                                      | 1/8 [22:30<2:37:35, 1350.77s/it]C:\Users\johns\AppData\Local\Temp\ipykernel_5792\659673755.py:51: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(
C:\Users\johns\anaconda3\envs\probstats\lib\site-packages\sklearn\model_selection\_split.py:909: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
[I 2023-04-08 22:07:10,915] A new study created in memory with name: no-name-4f3054bd-10d4-431f-b278-8dfd17c65bd3
[I 2023-04-08 22:07:26,578] Trial 1 finished with value: -1.2896334182849347 and parameters: {'max_depth': 33, 'num_leaves': 9, 'learning_rate': 0.002836852190209392, 'n_estimators': 21, 'reg_alpha': 0.0001349136291226134, 'reg_lambda': 242.5609572735276}. Best is trial 1 with value: -1.2896334182849347.
[I 2023-04-08 2

...

C:\Users\johns\anaconda3\envs\probstats\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\johns\anaconda3\envs\probstats\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\johns\AppData\Local\Temp\ipykernel_5792\659673755.py", line 48, in rmse
    y_pred = estimator.predict(X_test)
  File "C:\Users\johns\anaconda3\envs\probstats\lib\site-packages\lightgbm\sklearn.py", line 803, in predict
    return self._Booster.predict(X, raw_score=raw_score, start_iteration=start_iteration, num_iteration=num_iteration,
  File "C:\Users\johns\anaconda3\envs\probstats\lib\site-packages\lightgbm\basic.py", line 3538, in predict
    return predictor.predict(data, start_iteration, num_iteration,
  File "C:\Users\johns\anaconda3\envs\p

C:\Users\johns\anaconda3\envs\probstats\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\johns\anaconda3\envs\probstats\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\johns\AppData\Local\Temp\ipykernel_5792\659673755.py", line 48, in rmse
    y_pred = estimator.predict(X_test)
  File "C:\Users\johns\anaconda3\envs\probstats\lib\site-packages\lightgbm\sklearn.py", line 803, in predict
    return self._Booster.predict(X, raw_score=raw_score, start_iteration=start_iteration, num_iteration=num_iteration,
  File "C:\Users\johns\anaconda3\envs\probstats\lib\site-packages\lightgbm\basic.py", line 3538, in predict
    return predictor.predict(data, start_iteration, num_iteration,
  File "C:\Users\johns\anaconda3\envs\p

C:\Users\johns\anaconda3\envs\probstats\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\johns\anaconda3\envs\probstats\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\johns\AppData\Local\Temp\ipykernel_5792\659673755.py", line 48, in rmse
    y_pred = estimator.predict(X_test)
  File "C:\Users\johns\anaconda3\envs\probstats\lib\site-packages\lightgbm\sklearn.py", line 803, in predict
    return self._Booster.predict(X, raw_score=raw_score, start_iteration=start_iteration, num_iteration=num_iteration,
  File "C:\Users\johns\anaconda3\envs\probstats\lib\site-packages\lightgbm\basic.py", line 3538, in predict
    return predictor.predict(data, start_iteration, num_iteration,
  File "C:\Users\johns\anaconda3\envs\p

C:\Users\johns\anaconda3\envs\probstats\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\johns\anaconda3\envs\probstats\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\johns\AppData\Local\Temp\ipykernel_5792\659673755.py", line 48, in rmse
    y_pred = estimator.predict(X_test)
  File "C:\Users\johns\anaconda3\envs\probstats\lib\site-packages\lightgbm\sklearn.py", line 803, in predict
    return self._Booster.predict(X, raw_score=raw_score, start_iteration=start_iteration, num_iteration=num_iteration,
  File "C:\Users\johns\anaconda3\envs\probstats\lib\site-packages\lightgbm\basic.py", line 3538, in predict
    return predictor.predict(data, start_iteration, num_iteration,
  File "C:\Users\johns\anaconda3\envs\p

C:\Users\johns\anaconda3\envs\probstats\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\johns\anaconda3\envs\probstats\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\johns\AppData\Local\Temp\ipykernel_5792\659673755.py", line 48, in rmse
    y_pred = estimator.predict(X_test)
  File "C:\Users\johns\anaconda3\envs\probstats\lib\site-packages\lightgbm\sklearn.py", line 803, in predict
    return self._Booster.predict(X, raw_score=raw_score, start_iteration=start_iteration, num_iteration=num_iteration,
  File "C:\Users\johns\anaconda3\envs\probstats\lib\site-packages\lightgbm\basic.py", line 3538, in predict
    return predictor.predict(data, start_iteration, num_iteration,
  File "C:\Users\johns\anaconda3\envs\p

In [24]:
for key in models.keys():
    if models[key]:
        with open(f"./models/{key}.pkl", "wb") as f:
            pickle.dump(models[key].best_estimator_, f)